# Analysis for calculating score for Alerts


## Getting Data

This section loads sample data for 2015-05-21 records of activities, rules and actors.

In [5]:
import pandas as pd
import numpy as np
import math
from datetime import datetime

### Activities

In [6]:
activities_cols_names = ["id","body", "country", "country_code", "place_type",
                       "sub_region", "actor_id", "source", "share_count",
                       "in_reply_to_native_id", "created_at", "updated_at",
                       "klout", "native_id", "verb", "latitude", "longitude",
                       "sharing_activity_native_id", "region", "favorites_count",
                       "replies_count", "in_reply_to_screen_name", "link"]

In [8]:
activities_relevant_cols = ["actor_id", "source", "share_count",
                           "in_reply_to_native_id", "klout", "verb", "favorites_count",
                           "replies_count"]

In [11]:
activities = pd.read_csv('../../s3/2015-05-21-01-00-00-activities.csv', 
                         header=None, parse_dates=True,
                         names=activities_cols_names, index_col="id")

In [12]:
activities = activities[activities_relevant_cols] 

In [112]:
activities.head()

,actor_id,source,share_count,in_reply_to_native_id,klout,verb,favorites_count,replies_count
id,,,,,,,,
443640355408254904,2419819735,twitter,0,NaN,48,post,0,0
443640355408254905,2405587887,twitter,0,NaN,54,post,0,0
443640355408254907,2425857238,twitter,0,NaN,31,post,0,0
443640355408254908,2422152815,twitter,9,NaN,19,share,14,0
443640355408254909,2406991214,twitter,0,NaN,35,post,0,0


In [14]:
activities.describe()

,share_count,klout,favorites_count
count,1.280768e+06,1.255925e+06,1280865.000000
mean,1.878510e+12,4.836412e+13,3338.222079
std,1.062963e+15,5.392960e+15,13146.340361
min,0.000000e+00,1.000000e+01,0.000000
25%,1.000000e+00,2.900000e+01,0.000000
50%,6.000000e+01,4.000000e+01,54.000000
75%,7.090000e+02,4.500000e+01,782.000000
max,6.015506e+17,6.015513e+17,462900.000000


### Association Activities-Rules

In [15]:
activities_rules_cols_names = ["id", "rule_id", "activity_id", "created_at", "updated_at", "ignored"]

In [17]:
activities_rules = pd.read_csv('../../s3/2015-05-21-01-00-00-activities-rules.csv', 
                               parse_dates=True, names=activities_rules_cols_names, index_col="id")

In [18]:
activities_rules_relevant_cols = ["rule_id", "activity_id", "ignored"]

In [19]:
activities_rules = activities_rules[activities_rules_relevant_cols]

In [20]:
activities_rules.head()

,rule_id,activity_id,ignored
id,,,
108642970,5772,4.436404e+17,0
108642971,5052,4.436404e+17,0
108642972,5428,4.436404e+17,0
108642973,1506,4.436404e+17,0
108642974,5077,4.436404e+17,0


### Rules

In [22]:
rules = pd.read_csv('../../s3/rule.csv', index_col="id")

In [23]:
rules_relevant_cols = ["business_id","segment","volume", "category", "source", "type", "direct"]

In [24]:
rules = rules[rules_relevant_cols]

In [25]:
rules.head()

,business_id,segment,volume,category,source,type,direct
id,,,,,,,
1,144,1,0,2,twitter,TwitterRule,False
2,144,1,0,2,twitter,TwitterRule,False
3,144,1,0,2,twitter,TwitterRule,False
4,144,1,0,2,twitter,TwitterRule,False
5,144,1,0,2,twitter,TwitterRule,False


In [26]:
rules.describe()

,business_id,segment,volume,category,direct
count,4786.000000,4786.000000,4.464000e+03,4786.000000,4786
mean,314.314668,0.376097,2.326812e+05,3.243627,0.0215211
std,125.258811,0.484455,8.564815e+06,1.819087,0.1451287
min,1.000000,0.000000,0.000000e+00,0.000000,False
25%,262.000000,0.000000,1.600000e+01,2.000000,0
50%,354.000000,0.000000,2.970000e+02,3.000000,0
75%,412.000000,1.000000,3.140250e+03,5.000000,0
max,460.000000,1.000000,4.750714e+08,6.000000,True


### Actors

In [189]:
actors = pd.read_csv('../../s3/actors.csv', index_col="id")

In [119]:
actors.head()

,lang,favourites_count,statuses_count,friends_count,followers_count,listed_count
id,,,,,,
2419819735,en,NaN,16139,761,1065,71
2405587887,en,NaN,3043,1368,140,5
2425857238,en,NaN,15668,7,545,12
2422152815,en,NaN,171,57,30,0
2406991214,en,NaN,1341,306,190,7


In [82]:
actors.describe()

,favourites_count,statuses_count,friends_count,followers_count,listed_count
count,146204.000000,1278952.000000,1278953.000000,1278953.000000,1278650.000000
mean,2288.747558,29485.832285,1787.903539,17574.086707,147.851316
std,8553.651691,63277.951238,11217.130245,358911.933053,2558.850142
min,0.000000,0.000000,-438.000000,0.000000,0.000000
25%,15.000000,2208.000000,170.000000,157.000000,1.000000
50%,225.000000,9225.500000,396.000000,426.000000,3.000000
75%,1488.000000,29864.000000,1012.000000,1242.000000,14.000000
max,492244.000000,2051862.000000,1593447.000000,64199466.000000,821163.000000


## Computing scores

This section implements the functions for computing the scores and generates the scores for subsequent validation.

### Alert type - Influencer


- Legend:
    - k = klout
    - f = followers
    - l = listed count
    - v = verb
    - kw = klout weight
    - fw = followers count weight
    - lw = listed count weight

- Values:
    - kw = 30
    - fw = 40
    - lw = 30
    - v = (tweet=1, retweet=0.9)

**influencer_score** = (2-v) \* kw \* (k^2) / 10000 +
                        v \* fw \* log(f) / 20 +
                        v \* lw \* log(l) / 15


In [352]:
def generate_influencer_score(activity_id, k=None, f=None, l=None, verb=None):
    kw = 30.0
    fw = 40.0
    lw = 30.0
    
    if activity_id is not None:
        activity = activities.loc[str(activity_id)]
        actor = actors.loc[int(activity.actor_id)]
        if not activity.klout.is_integer():
            activity.klout = 0
    
    if k is None: k = float(activity.klout)
    if f is None: f = float(actor.followers_count + 1.0)
    if l is None: l = float(actor.listed_count + 1.0)
    
    v = 1.0 if verb=="post" else 0.9
    
    return int((2-v)*kw * float(k ** 2) / 10000 + \
                v * fw * math.log(f) / 20 + \
                v * lw * math.log(l) / 15)

In [353]:
for index, row in activities.head(30).iterrows():
    print "Generated score for activity {0}: {1}".format(index,generate_influencer_score(index))

Generated score for activity 443640355408254904: 27
Generated score for activity 443640355408254905: 21
Generated score for activity 443640355408254907: 19
Generated score for activity 443640355408254908: 7
Generated score for activity 443640355408254909: 17
Generated score for activity 443640355408254910: 13
Generated score for activity 443640355408254912: 26
Generated score for activity 443640355408254913: 6
Generated score for activity 443640355408254914: 29
Generated score for activity 443640355408254916: 22
Generated score for activity 443640355408254917: 11
Generated score for activity 443640355408254920: 28
Generated score for activity 443640355408254922: 16
Generated score for activity 443640355408254923: 21
Generated score for activity 443640355408254926: 11
Generated score for activity 443640355408254927: 46
Generated score for activity 443640355408254928: 15
Generated score for activity 443640355408254934: 55
Generated score for activity 443640355408254915: 40
Generated scor

Now testing with predefined values for evaluating limits:

In [354]:
generate_influencer_score(activity_id=None, k=99, f=50000000, l=50000000, verb='post')

100

In [355]:
generate_influencer_score(activity_id=None, k=90, f=50000000, l=50000000, verb='post')

95

In [356]:
generate_influencer_score(activity_id=None, k=99, f=50000000, l=50000000, verb='share')

96

In [357]:
generate_influencer_score(activity_id=None, k=90, f=50000000, l=50000000, verb='share')

90

In [358]:
generate_influencer_score(activity_id=None, k=70, f=5000000, l=5000000, verb='post')

76

In [359]:
generate_influencer_score(activity_id=None, k=70, f=5000000, l=5000000, verb='share')

71

In [360]:
generate_influencer_score(activity_id=None, k=50, f=5000000, l=5000000, verb='post')

69

In [361]:
generate_influencer_score(activity_id=None, k=50, f=5000000, l=5000000, verb='share')

63

In [362]:
generate_influencer_score(activity_id=None, k=50, f=1000000, l=1000000, verb='post')

62

In [363]:
generate_influencer_score(activity_id=None, k=50, f=1000000, l=1000000, verb='share')

57

### Alert type - Trends
* Reach of tweets
* segment: marketing (categories: range from 1 to 10) vs business(15% more important than Marketing)

**trend_score** = reach + segment




### Alert type - Popular Tweet
* Engagement of tweet
* Potential of engajamento

**popular_score** = engagement + potential
